In [2]:
import boto3
import pandas as pd
from botocore.exceptions import ClientError
import configparser
import json
import psycopg2

In [3]:
# read the config file
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))
KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')

CLUSTER_TYPE       = config.get("CLUSTER","CLUSTER_TYPE")
NUM_NODES          = config.get("CLUSTER","NUM_NODES")
NODE_TYPE          = config.get("CLUSTER","NODE_TYPE")

CLUSTER_IDENTIFIER = config.get("CLUSTER","CLUSTER_IDENTIFIER")
DB_NAME                 = config.get("CLUSTER","DB_NAME")
DB_USER            = config.get("CLUSTER","DB_USER")
DB_PASSWORD        = config.get("CLUSTER","DB_PASSWORD")
PORT               = config.get("CLUSTER","DB_PORT")

IAM_ROLE_NAME      = config.get("IAM_ROLE", "ARN")

In [4]:
df = pd.DataFrame({"Param":
                  ["CLUSTER_TYPE", "NUM_NODES", "NODE_TYPE", "CLUSTER_IDENTIFIER", 
                   "DB_NAME", "DB_USER", "DB_PASSWORD", "PORT", "IAM_ROLE_NAME"],
              "Value":
                  [CLUSTER_TYPE, NUM_NODES, NODE_TYPE, CLUSTER_IDENTIFIER, DB_NAME,
                   DB_USER, DB_PASSWORD, PORT, IAM_ROLE_NAME]
             })

In [5]:
df

,Param,Value
0,CLUSTER_TYPE,multi-node
1,NUM_NODES,4
2,NODE_TYPE,dc2.large
3,CLUSTER_IDENTIFIER,dwhCluster
4,DB_NAME,urdwh
5,DB_USER,urdwhuser
6,DB_PASSWORD,Passw0rd
7,PORT,5439
8,IAM_ROLE_NAME,URRole


### Create IAM Role and Attach Policy to it

In [6]:
def create_iam_role(iam, DWH_IAM_ROLE_NAME):
    # create an IAM role for redshift 
    # so that other aws services like s3 can be
    # accessed from redshift cluster
    
    try:
        # create IAM role
        print("1. Creating a new IAM Role") 
        dwhRole = iam.create_role(
            Path='/',
            RoleName=DWH_IAM_ROLE_NAME,
            Description = "Allows Redshift clusters to call AWS services on your behalf.",
            AssumeRolePolicyDocument=json.dumps(
                {'Statement': [{'Action': 'sts:AssumeRole',
                'Effect': 'Allow',
                'Principal': {'Service': 'redshift.amazonaws.com'}}],
                'Version': '2012-10-17'})
        )
        # Attach poliy to give s3 read access
        print("2. Attaching Policy")
        iam.attach_role_policy(RoleName=DWH_IAM_ROLE_NAME,
                            PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                            )['ResponseMetadata']['HTTPStatusCode']

        roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']

        print(roleArn)
    except Exception as e:
        print(e)

In [7]:
def createClients():
    ec2 = boto3.resource('ec2',
                     region_name='us-west-2', 
                     aws_access_key_id=KEY,
                     aws_secret_access_key=SECRET)

    s3 = boto3.resource('s3',
                         region_name='us-west-2', 
                         aws_access_key_id=KEY,
                         aws_secret_access_key=SECRET)

    iam = boto3.client('iam',
                         region_name='us-west-2', 
                         aws_access_key_id=KEY,
                         aws_secret_access_key=SECRET)

    redshift = boto3.client('redshift',
                         region_name='us-west-2', 
                         aws_access_key_id=KEY,
                         aws_secret_access_key=SECRET)
    
    roleArn = create_iam_role(iam, IAM_ROLE_NAME)
    return redshift,roleArn, ec2, s3

### Create Redshift Cluster

In [8]:
def create_cluster(redshift, roleArn, CLUSTER_TYPE, NODE_TYPE, NUM_NODES, 
                   DB_NAME, CLUSTER_IDENTIFIER, DB_USER, DB_PASSWORD):
    try:
        redshift.create_cluster(        
            #HW
            ClusterType=CLUSTER_TYPE,
            NodeType=NODE_TYPE,
            NumberOfNodes=int(NUM_NODES),

            #Identifiers & Credentials
            DBName=DB_NAME,
            ClusterIdentifier=CLUSTER_IDENTIFIER,
            MasterUsername=DB_USER,
            MasterUserPassword=DB_PASSWORD,
            
            #Roles (for s3 access)
            IamRoles=[roleArn]  
        )
    except Exception as e:
        print(e)

In [9]:
def openPort():
    # get the cluster properties to open port 
    myClusterProps = redshift.describe_clusters(ClusterIdentifier=CLUSTER_IDENTIFIER)['Clusters'][0]
    ENDPOINT = myClusterProps['Endpoint']['Address']
    ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
    try:
        vpc = ec2.Vpc(id=myClusterProps['VpcId'])
        defaultSg = list(vpc.security_groups.all())[0]
        print(defaultSg)
        defaultSg.authorize_ingress(
            GroupName=defaultSg.group_name,
            CidrIp='0.0.0.0/0',
            IpProtocol='TCP',
            FromPort=int(PORT),
            ToPort=int(PORT)
        )
        
        return ENDPOINT,ROLE_ARN
    except Exception as e:
        print(e)
        return ENDPOINT,ROLE_ARN

In [10]:
### 1. Create Clients
redshift,roleArn,ec2, s3 = createClients()

1. Creating a new IAM Role
An error occurred (EntityAlreadyExists) when calling the CreateRole operation: Role with name URRole already exists.


In [11]:
print(roleArn)

None


In [42]:
### 2. Create the cluster
create_cluster(redshift, roleArn, CLUSTER_TYPE, NODE_TYPE, NUM_NODES, 
               DB_NAME, CLUSTER_IDENTIFIER, DB_USER, DB_PASSWORD)